In [20]:
import numpy as np
import os
from selenium import webdriver
from selenium.webdriver.common.by import By

# Scrap the data on the web

In [ ]:

# Initialisation du navigateur Chrome
driver = webdriver.Chrome()
driver.maximize_window()
# Ouvrir une page web
driver.get("https://mon-vie-via.businessfrance.fr/offres/recherche?latest=true")



In [ ]:
# https://mon-vie-via.businessfrance.fr/offres



In [17]:
# Charger le fichier
with open("test.txt", "r", encoding="utf-8") as f:
    lignes = [ligne.strip() for ligne in f.readlines()]  # Nettoyage initial des lignes

# Créer la liste des chunks en ignorant les lignes vides
chunks = [lignes[i] for i in range(len(lignes)) if lignes[i] != ""]


In [21]:
print(chunks)
print(len(chunks))

["Qu'est-ce que l'apprentissage automatique ?", "L'apprentissage automatique (ou machine learning) est une branche de l'intelligence artificielle qui permet aux systèmes d'apprendre à partir de données sans être explicitement programmés.", 'Comment fonctionne un réseau de neurones ?', 'Un réseau de neurones est un modèle inspiré du cerveau humain, composé de couches de neurones artificiels. Chaque neurone reçoit des entrées, leur applique une transformation mathématique, et transmet le résultat à la couche suivante.', 'Quelle est la capitale de la France ?', 'La capitale de la France est Paris, qui est également la ville la plus peuplée du pays.', 'Citez trois langages de programmation populaires.', "Trois langages de programmation populaires sont Python, JavaScript et Java. Python est souvent utilisé pour l'analyse de données et l'IA, tandis que JavaScript domine le développement web.", "Qu'est-ce que le réchauffement climatique ?", "Le réchauffement climatique désigne l'augmentation 

# Create chunks of the data from the web

In [ ]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

# Create the embeding of the data we scrapped

In [24]:
from openai import OpenAI

api_key = os.environ.get("API_KEY_OPENAI")
client = OpenAI(api_key=api_key)

def get_text_embedding(input):
    response = client.embeddings.create(
        input=input,
        model="text-embedding-3-small"
    )
    print(response.data)
    return response.data[0].embedding

text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

[Embedding(embedding=[-0.026806006208062172, -0.011892521753907204, -0.05067161098122597, -0.05296754091978073, 0.038265522569417953, -0.01657501421868801, 0.0059915753081440926, 0.01931401900947094, -0.021650230512022972, 0.05844555422663689, 0.005191020201891661, -0.04273654893040657, -0.003396065207198262, -0.042414311319589615, 0.0023752315901219845, -0.002942920895293355, 0.02996794693171978, -0.01910255290567875, 0.01597082056105137, -0.04475052282214165, 0.027208801358938217, -0.019122691825032234, 0.030411021783947945, -0.018437940627336502, 0.01304048765450716, -0.11576327681541443, -0.0053319986909627914, -0.0031065563671290874, -0.04648254066705704, -0.04185039922595024, -0.0064648594707250595, -0.022919034585356712, -0.0023135538212955, 0.028215788304805756, 0.05385369062423706, -0.00978791806846857, 0.0037762029096484184, -0.008564428426325321, -0.021146737039089203, 0.02020016871392727, 0.004319976083934307, -0.0035848754923790693, 0.04962434247136116, 0.04483108222484588

# Let's save our text_embeddings text

# Now let's ask a question and embed it

In [25]:
question = "Donne moi un livre de Victor Hugo"

In [26]:
question_embeddings = np.array([get_text_embedding(question)])

[Embedding(embedding=[-0.0026457824278622866, 0.010189076885581017, -0.02137266844511032, -0.035145748406648636, 0.020603327080607414, -0.07077936828136444, -0.017206968739628792, -0.002619981300085783, -0.018548624590039253, -0.04589775949716568, 0.001976127503439784, 0.0010291108628734946, -0.03353200852870941, -0.019533755257725716, -0.014833269640803337, 0.039517857134342194, 0.04833713173866272, -0.016719093546271324, 0.07708421349525452, 0.05948319286108017, -0.003895961679518223, 0.009447882883250713, -0.014927092008292675, -0.014110839925706387, 0.02820291556417942, -0.0037130084820091724, -0.03206838294863701, -0.003853741567581892, 0.05374128371477127, -0.010423632338643074, -0.000675518938805908, -0.023549340665340424, 0.007407252676784992, 0.008073389530181885, 0.0023267874494194984, 0.029403837397694588, 0.00699443556368351, 0.013210147619247437, 0.028803376480937004, 0.020528269931674004, -0.015208558179438114, -0.05385386943817139, 0.0618475116789341, -0.0002078745310427

# Comparison bewtween our embedding vector and our embedding question

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
def find_closest_embedding(
    question_embeddings: np.array,
    text_embeddings: np.ndarray,
    top_n: int = 3,
) -> tuple[np.ndarray, np.ndarray]:

    # Calculer les similarités avec tous les embeddings
    similarites = cosine_similarity(question_embeddings, text_embeddings)[0]

    # Récupérer les top_n index et scores
    top_index = np.argsort(similarites)[-top_n:][::-1]  # Tri décroissant
    top_scores = similarites[top_index]

    return top_scores, top_index

In [29]:
top_scores, top_index = find_closest_embedding(question_embeddings, text_embeddings)

print("Top 3 embeddings les plus proches:")
for score, idx in zip(top_scores, top_index):
    print(f"- Embedding {idx}: Score = {score:.4f}")
    print(chunks[idx])

Top 3 embeddings les plus proches:
- Embedding 13: Score = 0.5294
"Les Misérables" est un roman écrit par Victor Hugo, publié en 1862. Il raconte l'histoire de Jean Valjean, un ancien forçat, dans la France du XIXe siècle.
- Embedding 12: Score = 0.4574
Qui a écrit "Les Misérables" ?
- Embedding 32: Score = 0.2112
Quelle est la formule de l'eau ?
